In [1]:
import os
from os import path
import pandas as pd
import numpy as np
import pickle
#import text2emotion as te
import json
import csv

os.chdir('./Dati')
print("current directory is : " + os.getcwd())

current directory is : C:\Users\david\Desktop\Progetto\Twitter\Dati


# Data Twitter

Tutti i tweet inerenti al COVID sono stati estratti utilizzando il framework TWARC (da prompt) e un repository git (https://github.com/lopezbec/COVID19_Tweets_Dataset) nel quale erano presenti gli ID dei tweet da Marzo ad Ottobre, nel notebook annesso "Hydrate tweets-ID" sono presenti le richieste effettuate per ottenere tutti i dati necessari. 

## Creazione dataset tweet

filtri: tweet che contenevano "mask", "no covid", "conspiracy"

In [2]:
def dataset(path):
    dt = pd.read_csv(path)
    #dt = dt.sort_values(by=['retweet_count'], ascending=False)
    dt.drop(['in_reply_to_status_id','in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str',
                        'in_reply_to_screen_name', 'coordinates','lang','geo','withheld_copyright', 'withheld_in_countries', 
                        'withheld_scope','matching_rules', 'contributors', 'retweeted_status', 'quote_count', 
                       'reply_count', 'filter_level', 'scopes', 'current_user_retweet','place', 'display_text_range',
                        'quoted_status_permalink'], axis=1, inplace=True)
    #dt["full_text_text"] = dt["full_text"].str.lower()
    dt = dt[dt["full_text"].str.contains('mask')  | dt["full_text"].str.contains('no covid') | dt["full_text"].str.contains('nocovid') | dt["full_text"].str.contains('conspiracy')]
    return dt
    

In [ ]:
dt = dataset('./hydratate/output_ottobre_01-31.csv')
dt = dt.append(dataset('./hydratate/output_settembre_01-30.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_agosto_01-31.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_luglio_01-31.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_giugno_01-30.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_maggio_01-31.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_aprile_01-30.csv'), ignore_index=True)
dt = dt.append(dataset('./hydratate/output_marzo_01-31.csv'), ignore_index=True)


In [5]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103210 entries, 0 to 103209
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   created_at            103210 non-null  object 
 1   id                    103210 non-null  int64  
 2   id_str                103210 non-null  int64  
 3   full_text             103210 non-null  object 
 4   source                103210 non-null  object 
 5   truncated             103210 non-null  bool   
 6   user                  103210 non-null  object 
 7   quoted_status_id      22313 non-null   float64
 8   quoted_status_id_str  22313 non-null   float64
 9   is_quote_status       103210 non-null  bool   
 10  quoted_status         20906 non-null   object 
 11  retweet_count         103210 non-null  int64  
 12  favorite_count        103210 non-null  int64  
 13  entities              103210 non-null  object 
 14  extended_entities     30648 non-null   object 
 15  

In [14]:
print('mediana\t\t',dt['retweet_count'].median(),'\ntotale\t', dt['retweet_count'].sum()) 
print(dt['retweet_count'].describe())

mediana		 23.0 
totale	 17614084
count    103210.000000
mean        170.662571
std        2095.355670
min           0.000000
25%          14.000000
50%          23.000000
75%          57.000000
max      253267.000000
Name: retweet_count, dtype: float64


### Selezioniamo tutti gli utenti che hanno fatto più di un tweet 

In [16]:
tweet = dt[dt.duplicated(subset=['user'], keep=False)]
print("Dimensioni dataset tweet: ",len(tweet))

Dimensioni dataset tweet:  25118


### Estraiamo gli id degli utenti che hanno effettuato il tweet e li aggiungiamo come colonna al dataset

In [71]:
# extrazione degli id tweet originali
id_list = []
for tweets in tweet['user']:
    id_list.append(tweets.split(",")[0].split(":")[1])

id_list = np.array(id_list, dtype=np.int64)
print(len(id_list))
tweet['user_id_tweet'] = id_list

25118


### Sentiment Analysis sul testo dei tweet

In [ ]:
sentiment_tweet = []

for text in tweet['full_text']:
    sentiment_tweet.append(te.get_emotion(text))

print(len(sentiment_tweet))
tweet['sentiment_analysis_tweet'] = sentiment_tweet

### Salviamo il dataset come file .csv così da poterlo caricare da file

In [22]:
tweet.to_csv('./dataset_tweet.csv', index=False)

In [2]:
tweet = pd.read_csv('./dataset_tweet.csv')
tweet.head()

,created_at,id,id_str,full_text,source,truncated,user,quoted_status_id,quoted_status_id_str,is_quote_status,quoted_status,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,user_id_tweet,sentiment_analysis_tweet
0,Tue Oct 06 21:42:32 +0000 2020,1313595523010625536,1313595523010625536,"If you're still social distancing, washing you...","<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1201348645225930752, 'id_str': '1201348...",NaN,NaN,False,NaN,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
1,Sun Oct 18 13:09:42 +0000 2020,1317815115543175168,1317815115543175168,Where we’re at: Twitter has removed a tweet (s...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 225265639, 'id_str': '225265639', 'name...",NaN,NaN,False,NaN,23286,106720,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,225265639,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.2, ..."
2,Wed Oct 07 15:22:22 +0000 2020,1313862238927638528,1313862238927638528,BREAKING: CDC finds Arizona infection rates \r...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 1383272101, 'id_str': '1383272101', 'na...",NaN,NaN,False,NaN,18537,64536,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,1383272101,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.5, ..."
3,Thu Oct 15 00:22:30 +0000 2020,1316534881304928257,1316534881304928257,Tucker Carlson was just told directly by the C...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'id': 1055040127741890560, 'id_str': '1055040...",NaN,NaN,False,NaN,15521,30784,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,1055040127741890560,"{'Happy': 0.2, 'Angry': 0.0, 'Surprise': 0.0, ..."
4,Wed Oct 21 02:27:09 +0000 2020,1318740576464941056,1318740576464941056,Does it make sense now? Wear a mask! https://t...,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 1134423305790795776, 'id_str': '1134423...",NaN,NaN,False,NaN,15259,14952,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1318740538099748864, 'id_str...",False,False,False,1134423305790795776,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."


In [3]:
print(tweet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25118 entries, 0 to 25117
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   created_at                25118 non-null  object 
 1   id                        25118 non-null  int64  
 2   id_str                    25118 non-null  int64  
 3   full_text                 25118 non-null  object 
 4   source                    25118 non-null  object 
 5   truncated                 25118 non-null  bool   
 6   user                      25118 non-null  object 
 7   quoted_status_id          6140 non-null   float64
 8   quoted_status_id_str      6140 non-null   float64
 9   is_quote_status           25118 non-null  bool   
 10  quoted_status             5761 non-null   object 
 11  retweet_count             25118 non-null  int64  
 12  favorite_count            25118 non-null  int64  
 13  entities                  25118 non-null  object 
 14  extend

## Creazione dataset retweet

I dati dei retweet sono stati ottenuti usato il framework TWARC (usando tramite script da prompt) che tramite gli ID dei tweet venivano effettuate le richieste relative ai retweet di gni tweet del dataset. Limitazione di richiedere al massimo 90 retweet per ogni tweet. 

In [4]:
# funzione di conversione file .jsonl in .csv

def convert(json_filename, output_filename):
    keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
              "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
              "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
              "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
              "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
              "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
              "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
              "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
              "quoted_status_permalink"]
    tweets = []
    with jsonlines.open(json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            tweets.append(i)

    # Writes them out
    with open(output_filename, "w+", encoding='utf-8-sig') as output_file:
        d = csv.DictWriter(output_file, keyset)
        d.writeheader()
        d.writerows(tweets)

    print('DONE')
    
convert('retweets.jsonl', 'retweet2.csv')

DONE


In [6]:
retweet = pd.read_csv('./retweet.csv')

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (24,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data cleaning

In [26]:
retweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977856 entries, 0 to 977855
Data columns (total 40 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   created_at                 977856 non-null  object 
 1   id                         977856 non-null  int64  
 2   id_str                     977856 non-null  int64  
 3   full_text                  977856 non-null  object 
 4   source                     977755 non-null  object 
 5   truncated                  977856 non-null  bool   
 6   in_reply_to_status_id      0 non-null       float64
 7   in_reply_to_status_id_str  0 non-null       float64
 8   in_reply_to_user_id        0 non-null       float64
 9   in_reply_to_user_id_str    0 non-null       float64
 10  in_reply_to_screen_name    0 non-null       float64
 11  user                       977856 non-null  object 
 12  coordinates                0 non-null       float64
 13  place                      0 

In [9]:
print(len(retweet))
retweet.drop_duplicates(['id'], inplace=True)
print(len(retweet))

977856
977856


### Elimino le colonne con valori tutti null

In [18]:
delete = ['in_reply_to_status_id','in_reply_to_status_id_str','in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name',
         'coordinates', 'place','quoted_status', 'quote_count','reply_count', 'filter_level','matching_rules','current_user_retweet','scopes',
         'withheld_copyright','withheld_in_countries','withheld_scope','geo','contributors' ]
retweet.drop(delete, axis=1, inplace=True)

In [19]:
print(retweet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977856 entries, 0 to 977855
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   created_at               977856 non-null  object 
 1   id                       977856 non-null  int64  
 2   id_str                   977856 non-null  int64  
 3   full_text                977856 non-null  object 
 4   source                   977755 non-null  object 
 5   truncated                977856 non-null  bool   
 6   user                     977856 non-null  object 
 7   quoted_status_id         229660 non-null  float64
 8   quoted_status_id_str     229660 non-null  float64
 9   is_quote_status          977856 non-null  bool   
 10  retweeted_status         977856 non-null  object 
 11  retweet_count            977856 non-null  int64  
 12  favorite_count           977856 non-null  int64  
 13  entities                 977856 non-null  object 
 14  exte

### Estrazione degli ID degli account che ha effettuato il retweet 

In [ ]:
# extrazione degli id tweet originali
id_list = []
#name_list = []
for retweets in retweet['user']:
    id_list.append(retweets.split(",")[0].split(":")[1])
    #name_list.append(retweets.split(",")[3].split(":")[1])

id_list = np.array(id_list, dtype=np.int64)
print(len(id_list))

retweet['user_id'] = id_list

### Estrazione degli ID dei tweet che sono stati retwittati

In [ ]:
# extrazione degli id tweet originali
id_list = []
for retweets in retweet['retweeted_status']:
    id_list.append(retweets.split(",")[1].split(":")[1])

id_list = np.array(id_list, dtype=np.int64)
print(len(id_list))

# aggiunto colonna id_tweet_originale utile per fare il merge con il dataset tweet
retweet['id_tweet_original'] = id_list

### Salviamo il dataset come file .csv così da poterlo caricare da file

In [ ]:
retweet.to_csv('./dataset_retweet.csv', index=False)

In [4]:
retweet = pd.read_csv('./dataset_retweet.csv')
retweet

,created_at,id,id_str,full_text,source,truncated,user,quoted_status_id,quoted_status_id_str,is_quote_status,...,entities,extended_entities,favorited,retweeted,possibly_sensitive,lang,display_text_range,quoted_status_permalink,id_tweet_original,user_id
0,Sat Dec 12 20:15:42 +0000 2020,1337853656016842752,1337853656016842752,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1431997939, 'id_str': '1431997939', 'na...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1313595523010625536,1431997939
1,Fri Dec 11 07:37:42 +0000 2020,1337300512711061508,1337300512711061508,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1101940683152285696, 'id_str': '1101940...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1313595523010625536,1101940683152285696
2,Fri Dec 11 05:03:44 +0000 2020,1337261763281231874,1337261763281231874,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 731887819, 'id_str': '731887819', 'name...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1313595523010625536,731887819
3,Thu Dec 10 16:25:01 +0000 2020,1337070826080923650,1337070826080923650,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 981957170, 'id_str': '981957170', 'name...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1313595523010625536,981957170
4,Thu Dec 10 16:24:37 +0000 2020,1337070728739352577,1337070728739352577,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1267574114497245186, 'id_str': '1267574...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1313595523010625536,1267574114497245186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977851,Mon Mar 02 17:00:02 +0000 2020,1234523874135379968,1234523874135379968,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 2922247225, 'id_str': '2922247225', 'na...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1234177480941920256,2922247225
977852,Mon Mar 02 12:55:46 +0000 2020,1234462402369982467,1234462402369982467,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1069668871232212992, 'id_str': '1069668...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1234177480941920256,1069668871232212992
977853,Mon Mar 02 01:14:39 +0000 2020,1234285960956801024,1234285960956801024,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1179192171259318272, 'id_str': '1179192...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1234177480941920256,1179192171259318272
977854,Sun Mar 01 22:35:09 +0000 2020,1234245820163219457,1234245820163219457,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1165817164718399488, 'id_str': '1165817...",NaN,NaN,False,...,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,en,"[0, 140]",NaN,1234177480941920256,1165817164718399488


In [5]:
print(retweet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977856 entries, 0 to 977855
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   created_at               977856 non-null  object 
 1   id                       977856 non-null  int64  
 2   id_str                   977856 non-null  int64  
 3   full_text                977856 non-null  object 
 4   source                   977755 non-null  object 
 5   truncated                977856 non-null  bool   
 6   user                     977856 non-null  object 
 7   quoted_status_id         229660 non-null  float64
 8   quoted_status_id_str     229660 non-null  float64
 9   is_quote_status          977856 non-null  bool   
 10  retweeted_status         977856 non-null  object 
 11  retweet_count            977856 non-null  int64  
 12  favorite_count           977856 non-null  int64  
 13  entities                 977856 non-null  object 
 14  exte

# Dataset retweet-tweet

Effettuiamo il merge tra il dataset dei retweet e quello dei tweet con chiave l'ID tweet

In [28]:
dataset_completo = retweet.merge(tweet, left_on=['id_tweet_original'], right_on=['id'])
dataset_completo

,created_at_x,id_x,id_str_x,full_text_x,source_x,truncated_x,user_x,quoted_status_id_x,quoted_status_id_str_x,is_quote_status_x,...,retweet_count_y,favorite_count_y,entities_y,extended_entities_y,favorited_y,retweeted_y,possibly_sensitive_y,full_text_text,user_id_tweet,sentiment_analysis_tweet
0,Sat Dec 12 20:15:42 +0000 2020,1337853656016842752,1337853656016842752,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1431997939, 'id_str': '1431997939', 'na...",NaN,NaN,False,...,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,"if you're still social distancing, washing you...",1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
1,Fri Dec 11 07:37:42 +0000 2020,1337300512711061508,1337300512711061508,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1101940683152285696, 'id_str': '1101940...",NaN,NaN,False,...,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,"if you're still social distancing, washing you...",1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
2,Fri Dec 11 05:03:44 +0000 2020,1337261763281231874,1337261763281231874,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 731887819, 'id_str': '731887819', 'name...",NaN,NaN,False,...,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,"if you're still social distancing, washing you...",1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
3,Thu Dec 10 16:25:01 +0000 2020,1337070826080923650,1337070826080923650,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 981957170, 'id_str': '981957170', 'name...",NaN,NaN,False,...,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,"if you're still social distancing, washing you...",1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
4,Thu Dec 10 16:24:37 +0000 2020,1337070728739352577,1337070728739352577,RT @VoLinxx: If you're still social distancing...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'id': 1267574114497245186, 'id_str': '1267574...",NaN,NaN,False,...,34838,30940,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,False,False,NaN,"if you're still social distancing, washing you...",1201348645225930752,"{'Happy': 0.14, 'Angry': 0.0, 'Surprise': 0.14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977851,Mon Mar 02 17:00:02 +0000 2020,1234523874135379968,1234523874135379968,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 2922247225, 'id_str': '2922247225', 'na...",NaN,NaN,False,...,8,40,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1234177460159184897, 'id_str...",False,False,False,@surgeon_general people in china also didn't t...,1166404686817030144,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
977852,Mon Mar 02 12:55:46 +0000 2020,1234462402369982467,1234462402369982467,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1069668871232212992, 'id_str': '1069668...",NaN,NaN,False,...,8,40,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1234177460159184897, 'id_str...",False,False,False,@surgeon_general people in china also didn't t...,1166404686817030144,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
977853,Mon Mar 02 01:14:39 +0000 2020,1234285960956801024,1234285960956801024,RT @shiroihamusan: @Surgeon_General People in ...,"<a href=""http://twitter.com/download/android"" ...",False,"{'id': 1179192171259318272, 'id_str': '1179192...",NaN,NaN,False,...,8,40,"{'hashtags': [], 'symbols': [], 'u

## DATASET FINALE

Come dataset finale selezioniamo tutte le persone che hanno retwittato più di una volta tweet inerenti al covid 

In [29]:
dataset = dataset_completo[dataset_completo.duplicated(subset=['user_id'], keep=False)]
print(len(dataset))

667577


In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667577 entries, 18 to 977854
Data columns (total 44 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   created_at_x              667577 non-null  object 
 1   id_x                      667577 non-null  int64  
 2   id_str_x                  667577 non-null  int64  
 3   full_text_x               667577 non-null  object 
 4   source_x                  667489 non-null  object 
 5   truncated_x               667577 non-null  bool   
 6   user_x                    667577 non-null  object 
 7   quoted_status_id_x        169146 non-null  float64
 8   quoted_status_id_str_x    169146 non-null  float64
 9   is_quote_status_x         667577 non-null  bool   
 10  retweeted_status          667577 non-null  object 
 11  retweet_count_x           667577 non-null  int64  
 12  favorite_count_x          667577 non-null  int64  
 13  entities_x                667577 non-null  

### Salviamo il dataset come file .csv così da poterlo caricare da file

In [31]:
dataset.to_csv('./dataset_finale.csv', index=False)

## Questa è la parte relativa alla preparazione dei dati, prosegue nel notebook "Twitter Graph"